### Workaround for Rochester qubit45 
This code is a workaround to the qubit45 problem of backend device ibmq_rochester. It will avoid your circuit to use qubit45. Note that only 52 qubits will be available for your circuits.

In [1]:
# function to create a new coupling map 
def fixed_coupling_map(backend):
    """A coupling map that works around qubit 45.
    
    Parameters:
        backend (IBMQBackend): The Rochester backend.
        
    Returns:
        list: New coupling map.
    """
    if backend.name() != 'ibmq_rochester':
        raise Exception('Fix applies to Rochester only.')
        
    cmap = backend.configuration().coupling_map
    new_cmap = [edge for edge in cmap if 45 not in edge]
    print(new_cmap)
    fixed_cmap = []
    for c in new_cmap:
        tmp0 = c[0]
        tmp1 = c[1]
        if c[0] > 45:
            tmp0 -= 1
        if c[1] > 45:
            tmp1 -= 1
        fixed_cmap.append([tmp0, tmp1])
        
    return fixed_cmap 

In [6]:
from qiskit import IBMQ

 # Load account from disk

API_token = 'f0988ffce4b74bf048cd5cd010ac77a67529086b139f5917f01e3cba8a65248c454be689b29d8c5b193fa6fbf01b0656231ee462d0bde1b829e12854712b45d2'
IBMQ.save_account(API_token, overwrite = True)
IBMQ.load_account()
IBMQ.providers()

provider = IBMQ.get_provider(hub='ibm-q-wits', group='internal', project='physics')
provider.backends()

ibmqfactory.load_account:WARNING:2020-07-07 10:58:11,009: Credentials are already in use. The existing account in the session will be replaced.


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rochester') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('

In [7]:
# The backend must be 'ibmq_rochester'
backend = provider.get_backend('ibmq_rochester')

# Use the new coupling map 
fixed_cmap = fixed_coupling_map(backend)

# First you must transpile your circuit using the new coupling_map 
trans_circ = transpile(circ, None, coupling_map=fixed_cmap, basis_gates=['id', 'u1', 'u2', 'u3', 'cx'])

[[0, 1], [0, 5], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 6], [5, 0], [5, 9], [6, 4], [6, 13], [7, 8], [7, 16], [8, 7], [8, 9], [9, 5], [9, 8], [9, 10], [10, 9], [10, 11], [11, 10], [11, 12], [11, 17], [12, 11], [12, 13], [13, 6], [13, 12], [13, 14], [14, 13], [14, 15], [15, 14], [15, 18], [16, 7], [16, 19], [17, 11], [17, 23], [18, 15], [18, 27], [19, 16], [19, 20], [20, 19], [20, 21], [21, 20], [21, 22], [21, 28], [22, 21], [22, 23], [23, 17], [23, 22], [23, 24], [24, 23], [24, 25], [25, 24], [25, 26], [25, 29], [26, 25], [26, 27], [27, 18], [27, 26], [28, 21], [28, 32], [29, 25], [29, 36], [30, 31], [30, 39], [31, 30], [31, 32], [32, 28], [32, 31], [32, 33], [33, 32], [33, 34], [34, 33], [34, 35], [34, 40], [35, 34], [35, 36], [36, 29], [36, 35], [36, 37], [37, 36], [37, 38], [38, 37], [38, 41], [39, 30], [39, 42], [40, 34], [40, 46], [41, 38], [41, 50], [42, 39], [42, 43], [43, 42], [43, 44], [44, 43], [44, 51], [46, 40], [46, 47], [47, 46], [47, 48], [48, 47], [

NameError: name 'transpile' is not defined

In [ ]:
# Create the new layout without qubit 45 
layout=list(range(45))+ list(range(46,53))

# Then execute your circuit using the new intial_layout
job = execute(trans_circ, backend, initial_layout=layout)